In [1]:
from make_test_samples import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [9]:
def plot_spk(s, loc=1):
    x = s.get_spikes_non_empty()
    plt.plot(x, [loc] * x.shape[0], '|')
    
def match_dists(sample_distances, distance, eps=1e-4):
    absdiff = np.abs(sample_distances - distance)
    inds = np.argwhere(absdiff <= eps)
    return inds


In [ ]:
freqs = [15, 50, 100]
desired_distances = [0.05, 0.1, 0.2, 0.3]
n = 10 # Number of neurons to test
duration_ms = 500

In [41]:
conds = {(15, 0.05): {'shifts': [3, 5, 7, 9]},
         (15, 0.1): {'shifts': [10, 15, 20, 30]},
         (15, 0.2): {'shifts': [20, 40, 60, 80]},
         (15, 0.3): {'shifts': [150, 200, 300]},
         (50, 0.05): {'shifts': [2,3,4,5], 'frac_shifts': [0.8]},
         (50, 0.1): {'shifts': [6,7,8], 'frac_shifts': [0.8, 1]},
         (50, 0.2): {'shifts': [10, 15, 20], 'frac_shifts': [0.8, 1]},
         (50, 0.3): {'shifts': [150, 200, 300]},
         (100, 0.05): {'shifts': [1, 2, 3], 'frac_shifts': [0.6, 0.8]},
         (100, 0.1): {'shifts': [4, 5, 6], 'frac_shifts': [0.6, 0.8]},
         (100, 0.2): {'shifts': [7, 8, 9], 'frac_shifts': [0.8, 1]},
         (100, 0.3): {'shifts': [150, 200, 300]}
        }

neurons = {f: make_spk(f, duration_ms, n, uniform_freq=False) for f in freqs}
N = 100
multi_ind = pd.MultiIndex.from_product([freqs, desired_distances])
# distances = pd.DataFrame(columns=multi_ind)
# good = pd.DataFrame(np.zeros(multi_ind.shape), index=multi_ind)
distances = {cond: [] for cond in multi_ind}
good = {cond: [] for cond in multi_ind}
summary = pd.DataFrame(np.zeros((multi_ind.shape[0], 2)), columns=('mean_dist', 'good_samps'), index=multi_ind)
for f in freqs:
    for dist in desired_distances:
        params = conds[(f, dist)]
        for neuron in neurons[f]:
            shifted = multi_shift(neuron, **params, n=N)
            distances[(f, dist)] = np.array(calc_distance(shifted, neuron)['vector']).flatten()
            summary['mean_dist'][(f, dist)] = distances[(f, dist)].mean()
            summary['good_samps'][(f, dist)] = match_dists(distances[(f, dist)], dist, eps=1e-3).shape[0]
summary

mean_dist  good_samps
15  0.05   0.043259        13.0
    0.10   0.113450         4.0
    0.20   0.217912         3.0
    0.30   0.294230         2.0
50  0.05   0.070048        15.0
    0.10   0.134918         9.0
    0.20   0.210707         8.0
    0.30   0.302791         9.0
100 0.05   0.046265        18.0
    0.10   0.092593        14.0
    0.20   0.156466         7.0
    0.30   0.312706         9.0